In [ ]:
# CNN

In [ ]:
# Deep Learning 보충학습2
# tensorflow.keras.datasets.mminst 학습

import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Flatten, Input, Dense, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import numpy as np

# load_data()로 데이터셋을 로드
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 1) keras 내장 데이터셋 로드

# 로드된 데이터셋 확인
print(f"train set: {x_train.shape}, {y_train.shape}")
print(f"test set: {x_test.shape}, {y_test.shape}")

# 데이터 시각화
fig, axes = plt.subplots(3,5)
fig.set_size_inches(8,5)

for i in range(15):
  ax = axes[i//5, i%5]
  # // = 나눗셈 후 소수점을 버리고 몫만 남긴 값
  # % = 나눗셈 후 남는 나머지
  ax.imshow(x_train[i], cmap='gray')
  ax.axis('off')
  ax.set_title(str(y_train[i]))
  
plt.tight_layout()
plt.show()

#########################
# 2)데이터 전처리

# x_train 배열의 데이터 확인
print("x_train 배열의 데이터 확인")
print(x_train[0, 10:15, 10:15]) # (순서, 행, 열)

# 픽셀 값의 최소/최대 값 확인
# min_val = np.min(x_train)
# max_val = np.max(x_train)
min_val = x_train.min()
max_val = x_train.max()

# 데이터 정규화
x_train = x_train / x_train.max()

# 정규화 후 최소/최대 값 확인
print(f"\n[정규화 전] 최소값: {min_val}, 최대값: {max_val}")
print(f"[정규화 후] 최소값: {x_train.min()}, 최대값: {x_train.max()}")

# test 셋에도 정규화 동일 적용
x_test = x_test / x_train.max()

# 변환 후 x_train 배열의 데이터 확인
print("\n변환 후 x_train 배열의 데이터 확인")
print(x_train[0, 10:15, 10:15])

#########################
# 3) Flatten 레이어 (N차원 → 1차원) : 평탄화(28*28=784)

# reshape 함수 사용 평탄화
print(f"\n변경 전 shape: {x_train.shape}")
print(f"reshape로 이미지 축 평탄화 후 shape: {x_train.reshape(60000, -1).shape}")
# 60000은 샘플 수, 딥러닝을 하기 위해 쓰이는 이미지 관련 축만 평탄화 진행
# 왜냐하면 딥러닝 모델의 입력 데이터 형식은 (샘플 수, 특징 수)여야 하기 때문

# Flatten 레이어 사용 평탄화
print(f"\n변경 전 shape: {x_train.shape}")
print(f"Flatten 레이어로 이미지 축 평탄환 후 shape: {Flatten()(x_train).shape}\n")

#########################
# 4) 활성화 함수

# Dense 레이어를 먼저 추가하고 활성화 함수를 별도로 적용한 경우
example = Sequential([
  Dense(128),
  Activation('relu')
])

#########################
# 5) 입력과 출력
model = Sequential([
  Input(shape=(x_train.shape[1], x_train.shape[2])),
  Flatten(),
  Dense(256, activation='relu'),
  Dense(64, activation='relu'),
  Dense(32, activation='relu'),
  Dense(10, activation='softmax')
])

#########################
# 6) 손실함수

# 방법1) 이진 분류 (출력 노드 개수 = 1, sigmoid 인 경우)
example.compile(loss='binary_crossentropy')

# 방법2) 다중분류 y가 원 핫 벡터인 경우
example.compile(loss='categorical_crossentropy')

# 방법3) y가 원 핫 벡터가 아닌 경우
example.compile(loss='sparse_categorical_crossentropy')

#########################
# 7) 옵티마이저

# 방법1) 클래스 인스턴스로 지정
adam = Adam(learning_rate=0.001)
example.compile(optimizer=adam)

# 방법2) 문자열로 지정
example.compile(optimizer='adam')

#########################
# 8) 평가지표

# 방법1) 클래스 인스턴스로 지정
acc = SparseCategoricalAccuracy()
example.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=[acc]
)

# 방법2) 문자열로 지정
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy']
)

#########################
# 9) 훈련

history = model.fit(
  x_train, y_train,
  validation_data=(x_test, y_test),
  epochs=10
)

#########################
# 10) 평가

loss, acc = model.evaluate(x_test, y_test)
print("\n정확도:", acc)

#########################
# 11) 예측

predictions = model.predict(x_test)
print("\n0번 index에 대한 클래스 점수값")
print(f"{predictions[0]}")

#########################
# 12) n번 째 index에 대한 예측 클래스 출력

# 0번 index에 대한 예측 클래스 출력
print(f"\n0번 index에 대한 예측 클래스: {np.argmax(predictions[0])}")

# 첫10개 index에 대한 예측 클래스 출력
print(f"첫 10개 index에 대한 예측 클래스: {np.argmax(predictions[:10], axis=1)}")

#########################
# 13) 데이터 시각화

# y_true(정답), y_pred(예측값), predictions(x_test예측값), confidence(정확도)
def get_one_result(i):
  img = x_test[i]
  y_true = y_test[i]
  y_pred = np.argmax(predictions[i])
  confidence = 100*np.max(predictions[i])
  return img, y_true, y_pred, confidence

fig, axes = plt.subplots(3, 5)
fig.set_size_inches(12, 10)

for i in range(15):
  ax = axes[i//5, i%5]
  
  img, y_true, y_pred, confidence = get_one_result(i)
  
  # imshow 로 이미지 시각화
  ax.imshow(img, cmap='gray') # 이미지
  ax.set_xticks([])
  ax.set_yticks([])
  ax.set_title(f"True: {y_true}")
  ax.set_xlabel(f"Prediction: {y_pred}\nConfidence: ({confidence:.2f}%)")

plt.tight_layout()
plt.show()

In [ ]:
# 4-2 실습

# mnist 불러오기 + 시각화

from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

# 데이터 불러오기
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 데이터 형태 출력
print("학습용 이미지 크기:", X_train.shape)
print("테스트용 이미지 크기:", X_test.shape)

# 이미지 5개 출력
plt.figure(figsize=(10,2))
for i in range(5):
  plt.subplot(1, 5, i+1)
  plt.imshow(X_train[i], cmap='gray')
  plt.title(f"Label: {y_train[i]}")
  plt.axis("off")
plt.show()

In [ ]:
# 4-3 일반 분류 딥러닝 & 이미지 분류 딥러닝 구조와 처리 방식

# 1) 일반 분류

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

# 데이터 불러오기
X, y = load_iris(return_X_y=True)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 구성, 컴파일, 학습
model = Sequential([
  Input(shape=(X.shape[1],)),
  Dense(64, activation='relu'),
  Dense(3, activation='softmax')
])
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy']
)

history = model.fit(
  X_train_scaled, y_train,
  epochs=30,
  batch_size=8,
  validation_split=0.2,
  verbose=1
)

# 정확도 시각화
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label="validation")
plt.title("Iris Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.show()


In [ ]:
# 4-3 일반 분류 딥러닝 & 이미지 분류 딥러닝 구조와 처리 방식

# 2) 이미지 분류

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Input, Dense
import matplotlib.pyplot as plt

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1) / 255.0
X_test = X_test.reshape(-1, 28, 28, 1) / 255.0

model = Sequential([
  Input(shape=(28, 28, 1)),
  Conv2D(32, (3, 3), activation='relu'),
  MaxPooling2D(),
  Conv2D(64, (3, 3), activation='relu'),
  MaxPooling2D(),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')
])
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['acc']
)

history = model.fit(
  X_train, y_train,
  epochs=5,
  batch_size=64,
  validation_split=0.1,
  verbose=1
)

plt.plot(history.history['acc'], label="Train")
plt.plot(history.history['val_acc'], label="Validation")
plt.title("MNIST Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.show()

# 예측값 일부 시각화
pred = model.predict(X_test[:10])
pred_classes = np.argmax(pred, axis=1)

fig, axes = plt.subplots(2, 5)
fig.set_size_inches(12, 4)

for i in range(10):
  ax = axes[i//5, i%5]
  
  ax.imshow(X_test[i].reshape(28, 28), cmap='gray')
  ax.axis('off')
  ax.set_title(f"Pred: {pred_classes[i]}")
plt.tight_layout()
plt.show()